In [1]:
from urllib.request import urlopen
from bs4 import BeautifulSoup
import selenium
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait as wait
from selenium.webdriver.support import expected_conditions as EC
import time
import pandas as pd


In [2]:
# 앨범 곡 정보 df - 곡명, 수록곡 길이
music = pd.DataFrame({'곡ID' : [],
                      '곡명' : [],
                      '아티스트' : [],
                      '장르' : [],
                      '가사' : []})

In [3]:
# 앨범 차트 크롤링
url = "https://music.bugs.co.kr/genre/home"
html = urlopen(url)
bsObject = BeautifulSoup(html, "html.parser")

# 드라이버
options = webdriver.ChromeOptions()
options.add_experimental_option("excludeSwitches", ["enable-logging"])
driver = webdriver.Chrome(options=options, executable_path='chromedriver')

driver.get(url)


In [ ]:
genres_xpath = {'발라드':'//*[@id="container"]/section/div/ul/li[1]/ul/li[1]/a',
                '댄스/팝':'//*[@id="container"]/section/div/ul/li[1]/ul/li[2]/a',
                '랩/힙합':'//*[@id="container"]/section/div/ul/li[1]/ul/li[5]/a'
               }


    


# 차트 페이지로 in
for i in genres_xpath:
    search = driver.find_element(By.XPATH, genres_xpath[i])
    time.sleep(1)
    try:
        search.click()
    except Exception:
        continue
    time.sleep(1)

    # 차트 내 곡 수 
    songs_num = len(driver.find_elements(By.XPATH, '//*[@id="CHARTday"]/table/tbody/tr/td[4]/a'))
    # 곡 상세페이지로 in
    for ID in range(songs_num):
        album = driver.find_elements(By.XPATH, '//*[@id="CHARTday"]/table/tbody/tr/td[4]/a')[ID]
        time.sleep(1)
        try:
            album.click()
        except Exception:
            continue
        # 곡 정보 가져오기
        title = driver.find_element(By.XPATH, '//div[@class="innerContainer"]/h1').text
        isin = False
        for i in range(len(driver.find_elements(By.XPATH, '//*[@id="container"]/section[1]/div/div[1]/table/tbody/tr'))):
            if driver.find_elements(By.XPATH, '//*[@id="container"]/section[1]/div/div[1]/table/tbody/tr/th')[i].text == '아티스트':
                artist = driver.find_element(By.XPATH, '//*[@id="container"]/section[1]/div/div[1]/table/tbody/tr[{}]/td/a'.format(i+1)).text
                isin = True
        if not isin:
            artist = None
        genre = i
        try:
            track_lyrics = driver.find_element(By.XPATH, '//div[@class="lyricsContainer"]/xmp').text
        except Exception:
            track_lyrics = None
        # 데이터프레임으로 저장
        music.loc[ID] = [ID, title, artist, genre, track_lyrics]
        driver.back()
        time.sleep(1)
    driver.back()
    time.sleep(1)

        
        

        
    
    
    

# csv 파일로 저장
music = music.astype({'곡ID':'int', '곡명':'string', '아티스트':'string', '장르':'string', '가사':'string'})
music.to_csv('Music.csv', index=False)




print("파일 저장이 완료되었습니다.")

//*[@id="container"]/section/div/ul/li[1]/ul/li[1]/a
100
//*[@id="container"]/section/div/ul/li[1]/ul/li[2]/a
100
